In [1]:
%pip install scikit-learn

/bin/bash: /home/kevin.wang/miniconda3/envs/tf-gpu-py310/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import nrrd
import os
from sklearn.model_selection import train_test_split
print("------------------------------------------------------------------------------------------------")
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("------------------------------------------------------------------------------------------------")

2024-07-06 23:15:35.461363: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-06 23:15:36.033418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


------------------------------------------------------------------------------------------------
2.16.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
------------------------------------------------------------------------------------------------


In [3]:

def encoder_block(inputs, output_channels, lastlayer=False):
    """
    Two 3x3x3 convolutions with batch normalization and ReLU activation
    2x2x2 max pool
    """

    # 3x3x3 convolutions with ReLU activation
    x = tf.keras.layers.Conv3D(int(output_channels/2), kernel_size=3, strides=1, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # 2x2x2 max pool

    if not lastlayer:
        x_maxPool = tf.keras.layers.MaxPool3D(pool_size=2, strides=2, padding = 'same')(x)
    else:
        x_maxPool = x

    return x, x_maxPool

def decoder_block(inputs, skip_features, output_channels):

    # Upsampling with 2x2x2 filter
    x = tf.keras.layers.Conv3DTranspose(output_channels*2, kernel_size=2, strides=2, padding = 'same')(inputs)

# Concatenate the skip features
    x = tf.keras.layers.Concatenate()([x, skip_features])

    # 2 convolutions with 3x3 filter, batch normalization, ReLU activation
    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv3D(output_channels, kernel_size=3, strides=1, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    return x

def unet_3D():
    inputs = tf.keras.Input(shape=(64, 64, 64, 1,))

    e1_skip, e1_maxpool = encoder_block(inputs, 64)
    e2_skip, e2_maxpool = encoder_block(e1_maxpool, 128)
    e3_skip, e3_maxpool = encoder_block(e2_maxpool, 256)
    _, e4 = encoder_block(e3_maxpool, 512, True)

    decoder1 = decoder_block(e4, e3_skip, 256)
    decoder2 = decoder_block(decoder1, e2_skip, 128)
    decoder3 = decoder_block(decoder2, e1_skip, 64)

    outputs = tf.keras.layers.Conv3D(1, 1, strides = 1)(decoder3)
    outputs = tf.keras.layers.Activation('sigmoid')(outputs)

    model = tf.keras.models.Model(inputs = inputs,  outputs = outputs,  name = 'Unet3D')

    return model
    

In [4]:
def iou(y_true, y_pred, smooth=0.000000001):
    # yt = K.argmax(y_true, axis=2)
    # yp = K.argmax(y_pred, axis=2)
    # print(y_pred)
    #yp = y_pred[0]
    # yp[yp>=0.5]=1
    # yp[yp<0.5]=0
    yp = y_pred
    yp = tf.where(yp >= 0.5, tf.ones_like(yp), yp)
    yp = tf.where(yp < 0.5, tf.zeros_like(yp), yp)
    yp = K.cast(yp, np.float32)

    yt = K.cast(y_true, np.float32)

    # print(yt.shape)
    # print(yp.shape)
    
    intersection = K.sum(yt * yp)
    union = K.sum(yt) + K.sum(yp)
    # intersection = K.sum(yt * yp, axis=1)
    # union = K.sum(yt, axis=1) + K.sum(yp, axis=1)
    return (intersection + smooth) / (union-intersection+smooth)

In [5]:
model = unet_3D()
# model.summary()

print("compiling model")
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='dice', metrics=[iou])#, metrics=[iou])

2024-07-06 23:15:37.119583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10089 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:65:00.0, compute capability: 8.9


compiling model


In [6]:
trainlist = os.listdir("./gt")

In [7]:
import ast
print("loading inputs")

number_inputs = len(trainlist)

X, _ =  nrrd.read("inputs/" + trainlist[0])
X = np.array([X]).astype(np.float32)
X = np.expand_dims(X, -1)
for i in range(1, number_inputs):

    try:
        volume, _ =  nrrd.read("inputs/" + trainlist[i])
        volume = np.array([volume])
        volume = np.expand_dims(volume, -1)
        X = np.concatenate((X, volume), axis=0)
    except:
        print("skipping " + trainlist[i])

print("loading ground truths")

valid_samples = [trainlist[0]]

y, _ =  nrrd.read("gt/" + trainlist[0])
y = np.array([y])
y = np.expand_dims(y, axis=-1)

for i in range(1, number_inputs):
    try:
        volume, _ =  nrrd.read("gt/" + trainlist[i])
        volume = np.array([volume])
        volume = np.expand_dims(volume, axis=-1)
        y = np.concatenate((y, volume), axis=0)
        valid_samples.append(trainlist[i])
    except:
        print("skipping " + trainlist[i])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1)


loading inputs
skipping 63_volume_13_aug_1.nrrd
skipping 5_volume_22_aug_1.nrrd
skipping 31_volume_14_aug_1.nrrd
skipping 31_volume_10_aug_0.nrrd
skipping 31_volume_10_aug_2.nrrd
skipping 5_volume_19_aug_1.nrrd
skipping 91_volume_10_aug_0.nrrd
skipping 84_volume_7_aug_0.nrrd
skipping 31_volume_12_aug_1.nrrd
skipping 63_volume_12_aug_1.nrrd
skipping 5_volume_21_aug_1.nrrd
skipping 63_volume_8_aug_0.nrrd
skipping 63_volume_6_aug_2.nrrd
skipping 91_volume_9_aug_0.nrrd
skipping 31_volume_9_aug_2.nrrd
skipping 63_volume_11_aug_1.nrrd
skipping 5_volume_20_aug_1.nrrd
skipping 31_volume_13_aug_1.nrrd
loading ground truths
skipping 63_volume_13_aug_1.nrrd
skipping 5_volume_22_aug_1.nrrd
skipping 31_volume_14_aug_1.nrrd
skipping 31_volume_10_aug_0.nrrd
skipping 31_volume_10_aug_2.nrrd
skipping 5_volume_19_aug_1.nrrd
skipping 91_volume_10_aug_0.nrrd
skipping 84_volume_7_aug_0.nrrd
skipping 31_volume_12_aug_1.nrrd
skipping 63_volume_12_aug_1.nrrd
skipping 5_volume_21_aug_1.nrrd
skipping 63_volume_

In [8]:
# Checkpoint Saving
checkpoint_path = "./checkpoints/cp-{epoch:04d}.weights.h5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1,
                                                 save_weights_only=True, save_freq='epoch') #save_freq=1850)


print("---------------- fitting model ---------------------")
model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=2, epochs=200, callbacks = [cp_callback])

---------------- fitting model ---------------------
Epoch 1/200


I0000 00:00:1720322539.089876  336513 service.cc:145] XLA service 0x73f2a8034120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720322539.089963  336513 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-07-06 23:22:19.269420: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-06 23:22:19.967097: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


  1/521 ━━━━━━━━━━━━━━━━━━━━ 3:34:58 25s/step - iou: 1.1696e-04 - loss: 0.9998

I0000 00:00:1720322558.163366  336513 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - iou: 0.0065 - loss: 0.9956
Epoch 1: saving model to ./checkpoints/cp-0001.weights.h5
521/521 ━━━━━━━━━━━━━━━━━━━━ 118s 178ms/step - iou: 0.0065 - loss: 0.9956 - val_iou: 0.0460 - val_loss: 0.9943
Epoch 2/200
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - iou: 0.0314 - loss: 0.9868
Epoch 2: saving model to ./checkpoints/cp-0002.weights.h5
521/521 ━━━━━━━━━━━━━━━━━━━━ 80s 154ms/step - iou: 0.0314 - loss: 0.9868 - val_iou: 0.1442 - val_loss: 0.9751
Epoch 3/200
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - iou: 0.0539 - loss: 0.9700
Epoch 3: saving model to ./checkpoints/cp-0003.weights.h5
521/521 ━━━━━━━━━━━━━━━━━━━━ 80s 154ms/step - iou: 0.0539 - loss: 0.9700 - val_iou: 0.1934 - val_loss: 0.9139
Epoch 4/200
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - iou: 0.1139 - loss: 0.9006
Epoch 4: saving model to ./checkpoints/cp-0004.weights.h5
521/521 ━━━━━━━━━━━━━━━━━━━━ 80s 154ms/step - iou: 0.1139 - loss: 0.9005 - val_iou: 0.1763 - val_loss: 0.8264
Epo